---
---

## DHS2019_Feature_Engineering


#### PROBLEM STATEMENT
---


Email Marketing is still the most successful marketing channel and the essential element of any digital marketing strategy. Marketers spend a lot of time in writing that perfect email, labouring over each word, catchy layouts on multiple devices to get them best in-industry open rates & click rates.

How can I build my campaign to increase the click-through rates of email? - a question that is often heard when marketers are creating their email marketing plans.

Can we optimize our email marketing campaigns with Data Science?

It's time to unlock marketing potential and build some exceptional data-science products for email marketing.

Analytics Vidhya sends out marketing emailers for various events such as conferences, hackathons, etc. We have provided a sample of user-email interaction data from July 2017 to December 2017. You are required to predict the click probability of links inside a mailer for email campaigns from January 2018 to March 2018.


#### **Contest URL: https://datahack.analyticsvidhya.com/contest/workshop_lord-of-the-machines-data-science-hackath/**



---
---

In [3]:
!pip3 install lightgbm

In [ ]:
import numpy as np
import pandas as pd
import time
from sklearn import model_selection, preprocessing, metrics, ensemble
import lightgbm as lgb

In [ ]:
# read the datasets
train_df = pd.read_csv("dataset/train.csv")
test_df = pd.read_csv("dataset/test.csv")
campaign_df = pd.read_csv("dataset/campaign_data.csv")

# create another column to label train data as 1 and test data as 0
train_df["train_set"] = 1
test_df["train_set"] = 0
print("Train and test shape : ", train_df.shape, test_df.shape)

In [ ]:
# view the train data
train_df.head()

In [ ]:
# view the test data
test_df.head()

In [ ]:
# view the campaign data
campaign_df.head(3)

In [ ]:
# convert send date to date time format in both train and test data
train_df["send_date"] = pd.to_datetime(train_df["send_date"], format="%d-%m-%Y %H:%M")
test_df["send_date"] = pd.to_datetime(test_df["send_date"], format="%d-%m-%Y %H:%M")

# create new featrure oridnal date 
# The timestamp returns a long integer containing the number of 
# seconds between the Unix Epoch (January 1, 1970, 00:00:00 GMT) and the time specified.
train_df["ordinal_date"] = train_df["send_date"].apply(lambda x: time.mktime(x.timetuple()))
test_df["ordinal_date"] = test_df["send_date"].apply(lambda x: time.mktime(x.timetuple()))

# sort the dataframes using oridnal date
train_df = train_df.sort_values(by="ordinal_date").reset_index(drop=True)
test_df = test_df.sort_values(by="ordinal_date").reset_index(drop=True)

# add more features to train and test data by merging it with campaign data
train_df = pd.merge(train_df, campaign_df, on="campaign_id")
test_df = pd.merge(test_df, campaign_df, on="campaign_id")

In [ ]:
for c in ["communication_type"]:
    lbl = preprocessing.LabelEncoder()
    lbl.fit(list(train_df[c].values.astype('str')) + list(test_df[c].values.astype('str')))
    train_df[c] = lbl.transform(list(train_df[c].values.astype('str')))
    test_df[c] = lbl.transform(list(test_df[c].values.astype('str')))

train_df.head()

### Baseline Model

In [ ]:
# define a function to train the lightGBM model
def runLGB(train_X, train_y, test_X, test_y=None, test_X2=None): 
    params = {}
    params["objective"] = "binary"
    params['metric'] = 'auc'
    params["max_depth"] = 8
    params["min_data_in_leaf"] = 1
    params["learning_rate"] = 0.001
    params["bagging_fraction"] = 0.7
    params["feature_fraction"] = 0.7
    params["bagging_freq"] = 5
    params["bagging_seed"] = 0
    params["verbosity"] = -1
    num_rounds = 10000

    plst = list(params.items())
    lgtrain = lgb.Dataset(train_X, label=train_y)

    if test_y is not None:
        lgtest = lgb.Dataset(test_X, label=test_y)
        model = lgb.train(params, lgtrain, num_rounds, valid_sets=[lgtest], early_stopping_rounds=100, verbose_eval=50)
    else:
        lgtest = lgb.DMatrix(test_X)
        model = lgb.train(params, lgtrain, num_rounds)

    pred_test_y = model.predict(test_X, num_iteration=model.best_iteration)
    pred_test_y2 = model.predict(test_X2, num_iteration=model.best_iteration)

    loss = 0
    if test_y is not None:
        loss = metrics.roc_auc_score(test_y, pred_test_y)
        print(loss)
        return pred_test_y, loss, pred_test_y2
    else:
        return pred_test_y, loss, pred_test_y2

In [ ]:
# filter the columns that you want to use to train the model
usecols = ["user_id", "communication_type", "total_links", "no_of_internal_links", "no_of_images", "no_of_sections"]
train_X = train_df[usecols]
test_X = test_df[usecols]
train_y = train_df["is_click"].values

In [ ]:
# GroupKFold with n_splits=5
cv_scores = []
pred_test_full = 0
kf = model_selection.GroupKFold(n_splits=5)
for dev_index, val_index in kf.split(train_df, train_df["is_click"].values, train_df["campaign_id"].values):
    # print the campaign IDs of the current training set
    print(train_df["campaign_id"].loc[val_index].unique())
    dev_X, val_X = train_X.loc[dev_index,:], train_X.loc[val_index,:]
    dev_y, val_y = train_y[dev_index], train_y[val_index]
    
    # train a lgb model
    pred_val, loss, pred_test = runLGB(dev_X, dev_y, val_X, val_y, test_X)
    cv_scores.append(loss)
    pred_test_full += pred_test
    print(cv_scores)
pred_test_full /= 5.

In [ ]:
# save the calculated results in the csv file and submit it on the contest page to get the results
# to create the sample submission file refer to the sample submission file as given on the contest page
sub_df = pd.DataFrame({"id":test_df["id"].values})
sub_df["is_click"] = pred_test_full
sub_df.to_csv("sub1.csv", index=False)



In [ ]:
sub_df.head()

### Check if you can remove some variables and still the model results are similar

If you think some of the variables above might be not add much value, then remove them and re-run the models

In [ ]:
### Add the columns in the 'usecols' below
# ******* EXERCISE 1 **********
usecols = [] 
train_X = train_df[usecols]
test_X = test_df[usecols]
train_y = train_df["is_click"].values

In [ ]:
# GroupKFold with n_splits=5
cv_scores = []
pred_test_full = 0
kf = model_selection.GroupKFold(n_splits=5)
for dev_index, val_index in kf.split(train_df, train_df["is_click"].values, train_df["campaign_id"].values):
    # print the campaign IDs of the current training set
    print(train_df["campaign_id"].loc[val_index].unique())
    dev_X, val_X = train_X.loc[dev_index,:], train_X.loc[val_index,:]
    dev_y, val_y = train_y[dev_index], train_y[val_index]
    
    # train a lgb  model
    pred_val, loss, pred_test = runLGB(dev_X, dev_y, val_X, val_y, test_X)
    pred_test_full += pred_test
    cv_scores.append(loss)
    print(cv_scores)
pred_test_full /= 5.

In [ ]:
# submit the results to check the scores
sub_df = pd.DataFrame({"id":test_df["id"].values})
sub_df["is_click"] = pred_test_full
sub_df.to_csv("sub2.csv", index=False)



In [ ]:
sub_df.head()

### Temporal variables 

Which temporal variables can be created for this problem? Please extract the same and use them to rebuild the models.

In [ ]:

# ****** EXERCISE 2*******
## Create temporal variables here based on the date column

In [ ]:
## Use the previously selected variables and also the temporal variables
usecols = [] 
train_X = train_df[usecols]
test_X = test_df[usecols]
train_y = train_df["is_click"].values

In [ ]:
cv_scores = []
kf = model_selection.GroupKFold(n_splits=5)
for dev_index, val_index in kf.split(train_df, train_df["is_click"].values, train_df["campaign_id"].values):
    print(train_df["campaign_id"].loc[val_index].unique())
    dev_X, val_X = train_X.loc[dev_index,:], train_X.loc[val_index,:]
    dev_y, val_y = train_y[dev_index], train_y[val_index]
    
    pred_val, loss, pred_test = runLGB(dev_X, dev_y, val_X, val_y, test_X)
    cv_scores.append(loss)
    print(cv_scores)

### Target Encoding

In [ ]:
def getDVEncodeVar(compute_df, target_df, var_name, target_var="is_click", min_cutoff=1):
    if type(var_name) != type([]):
        var_name = [var_name]
    grouped_df = target_df.groupby(var_name)[target_var].agg(["mean"]).reset_index()
    grouped_df.columns = var_name + ["mean_value"]
    merged_df = pd.merge(compute_df, grouped_df, how="left", on=var_name)
    merged_df.fillna(np.mean(target_df[target_var].values), inplace=True)
    return list(merged_df["mean_value"])


cols_to_use = []


kf = model_selection.GroupKFold(n_splits=5)
#### add columns to target encode
# ****** EXERCISE 3*******
for col in ["Add the columns to target encode here"]: ## [["id"], ["id", "date"]]
        train_enc_values = np.zeros(train_df.shape[0])
        test_enc_values = 0
        for dev_index, val_index in kf.split(train_df, train_df["is_click"].values, train_df["campaign_id"].values):
            dev_X, val_X = train_df.loc[dev_index], train_df.loc[val_index]
            train_enc_values[val_index] = np.array( getDVEncodeVar(val_X[col], dev_X, col, 'is_click'))
            test_enc_values += np.array( getDVEncodeVar(test_df[col], dev_X, col, 'is_click'))
        test_enc_values /= 5.
        if isinstance(col, list):
            col = "_".join(col)
        train_df[col + "_enc"] = train_enc_values
        test_df[col + "_enc"] = test_enc_values
        cols_to_use.append(col + "_enc")

In [ ]:
#### EXERCISE 4
usecols = ["Add the previously selected cols here"] + cols_to_use
print(usecols)
train_X = train_df[usecols]
test_X = test_df[usecols]
train_y = train_df["is_click"].values

In [ ]:
cv_scores = []
pred_test_full = 0
kf = model_selection.GroupKFold(n_splits=5)
for dev_index, val_index in kf.split(train_df, train_df["is_click"].values, train_df["campaign_id"].values):
    print(train_df["campaign_id"].loc[val_index].unique())
    dev_X, val_X = train_X.loc[dev_index,:], train_X.loc[val_index,:]
    dev_y, val_y = train_y[dev_index], train_y[val_index]
    
    pred_val, loss, pred_test = runLGB(dev_X, dev_y, val_X, val_y, test_X)
    cv_scores.append(loss)
    pred_test_full += pred_test
    print(cv_scores)
pred_test_full /= 5.

In [ ]:
sub_df = pd.DataFrame({"id":test_df["id"].values})
sub_df["is_click"] = pred_test_full
sub_df.to_csv("sub3.csv", index=False)



### Target encoding with open as target

In [ ]:
cols_to_use = []
kf = model_selection.GroupKFold(n_splits=5)
#### EXERCISE 5
for col in ["Add the columns to be target encoded here"]: # [["id"], ["id", "date"]]
        train_enc_values = np.zeros(train_df.shape[0])
        test_enc_values = 0
        for dev_index, val_index in kf.split(train_df, train_df["is_click"].values, train_df["campaign_id"].values):
            dev_X, val_X = train_df.loc[dev_index], train_df.loc[val_index]
            train_enc_values[val_index] = np.array( getDVEncodeVar(val_X[col], dev_X, col, 'is_open'))
            test_enc_values += np.array( getDVEncodeVar(test_df[col], dev_X, col, 'is_open'))
        test_enc_values /= 5.
        if isinstance(col, list):
            col = "_".join(col)
        train_df[col + "_open_enc"] = train_enc_values
        test_df[col + "_open_enc"] = test_enc_values
        cols_to_use.append(col + "_open_enc")

In [ ]:
#### EXERCISE 6
usecols = ["Previously used cols for model building"] + cols_to_use
print(usecols)
train_X = train_df[usecols]
test_X = test_df[usecols]
train_y = train_df["is_click"].values

In [ ]:
cv_scores = []
pred_test_full = 0
kf = model_selection.GroupKFold(n_splits=5)
for dev_index, val_index in kf.split(train_df, train_df["is_click"].values, train_df["campaign_id"].values):
    print(train_df["campaign_id"].loc[val_index].unique())
    dev_X, val_X = train_X.loc[dev_index,:], train_X.loc[val_index,:]
    dev_y, val_y = train_y[dev_index], train_y[val_index]
    
    pred_val, loss, pred_test = runLGB(dev_X, dev_y, val_X, val_y, test_X)
    cv_scores.append(loss)
    pred_test_full += pred_test
    print(cv_scores)
pred_test_full /= 5.

In [ ]:
sub_df = pd.DataFrame({"id":test_df["id"].values})
sub_df["is_click"] = pred_test_full
sub_df.to_csv("sub4.csv", index=False)



### Count for each target encoding variables

In [ ]:
def getCountVar(compute_df, count_df, var_name, count_var="v1"):
    grouped_df = count_df.groupby(var_name)[count_var].agg('count').reset_index()
    grouped_df.columns = var_name + ["var_count"]

    merged_df = pd.merge(compute_df, grouped_df, how="left", on=var_name)
    merged_df.fillna(np.mean(grouped_df["var_count"].values), inplace=True)
    return list(merged_df["var_count"])

cols_to_use = []
kf = model_selection.GroupKFold(n_splits=5)
#### EXERCISE 7
for col in ["Add the columns for count encoding here"]:
        train_enc_values = np.zeros(train_df.shape[0])
        test_enc_values = 0
        for dev_index, val_index in kf.split(train_df, train_df["is_click"].values, train_df["campaign_id"].values):
            dev_X, val_X = train_df.loc[dev_index], train_df.loc[val_index]
            train_enc_values[val_index] = np.array( getCountVar(val_X[col], dev_X, col, 'is_open'))
            test_enc_values += np.array( getCountVar(test_df[col], dev_X, col, 'is_open'))
        test_enc_values /= 5.
        if isinstance(col, list):
            col = "_".join(col)
        train_df[col + "_count"] = train_enc_values
        test_df[col + "_count"] = test_enc_values
        cols_to_use.append(col + "_count")

In [ ]:
#### EXERCISE 8
usecols = ["Previously used cols for model building"] + cols_to_use
print(usecols)
train_X = train_df[usecols]
test_X = test_df[usecols]
train_y = train_df["is_click"].values

In [ ]:
cv_scores = []
pred_test_full = 0
kf = model_selection.GroupKFold(n_splits=5)
for dev_index, val_index in kf.split(train_df, train_df["is_click"].values, train_df["campaign_id"].values):
    print(train_df["campaign_id"].loc[val_index].unique())
    dev_X, val_X = train_X.loc[dev_index,:], train_X.loc[val_index,:]
    dev_y, val_y = train_y[dev_index], train_y[val_index]
    
    pred_val, loss, pred_test = runLGB(dev_X, dev_y, val_X, val_y, test_X)
    cv_scores.append(loss)
    pred_test_full += pred_test
    print(cv_scores)
pred_test_full /= 5.

In [ ]:
sub_df = pd.DataFrame({"id":test_df["id"].values})
sub_df["is_click"] = pred_test_full
sub_df.to_csv("sub5.csv", index=False)



### More Features

So far we have created features based on interactions and textbook ideas. Now let us wear the business hat and come up with new features

In [1]:
### EXERCISE 9
### Add more features here

In [ ]:
### Add more features

In [ ]:
usecols = ['Enter all the features to use here']
train_X = train_df[usecols]
test_X = test_df[usecols]
train_y = train_df["is_click"].values
test_id = test_df["id"].values
print(usecols)

In [ ]:
cv_scores = []
pred_test_full = 0
kf = model_selection.GroupKFold(n_splits=5)
for dev_index, val_index in kf.split(train_df, train_df["is_click"].values, train_df["campaign_id"].values):
    print(train_df["campaign_id"].loc[val_index].unique())
    dev_X, val_X = train_X.loc[dev_index,:], train_X.loc[val_index,:]
    dev_y, val_y = train_y[dev_index], train_y[val_index]
    
    pred_val, loss, pred_test = runLGB(dev_X, dev_y, val_X, val_y, test_X)
    pred_test_full += pred_test
    cv_scores.append(loss)
    print(cv_scores)
pred_test_full /= 5.

In [ ]:
sub_df = pd.DataFrame({"id":test_df["id"].values})
sub_df["is_click"] = pred_test_full
sub_df.to_csv("sub6.csv", index=False)

